In [1]:
from pynput import mouse
from pynput import keyboard
from time import sleep
import math

mctl = mouse.Controller()
kctl = keyboard.Controller()

In [2]:
STANDARD_INPUT_SLEEP = 0.05
DEFAULT_ACTION_TIME = 4
def lclick(x, y):
    mctl.position = (x, y)
    mctl.click(mouse.Button.left)
    sleep(STANDARD_INPUT_SLEEP)
def rclick(x, y):
    mctl.position = (x, y)
    mctl.click(mouse.Button.right)
    sleep(STANDARD_INPUT_SLEEP)
def ldrag(x1, y1, x2, y2):
    mctl.position = (x1, y1)
    mctl.press(mouse.Button.left)
    sleep(STANDARD_INPUT_SLEEP)
    mctl.position = (x2, y2)
    sleep(STANDARD_INPUT_SLEEP)
    mctl.release(mouse.Button.left)
    sleep(STANDARD_INPUT_SLEEP)
def scroll(x, y, dy):
    mctl.position = (x, y)
    mctl.scroll(0, dy)
    sleep(STANDARD_INPUT_SLEEP + 0.2)

The following block defines the camera manager, which models the ingame state of the camera and programmatically points it wherever requested by the code.

In [3]:
class CameraManager:
    def __init__(self, zoom=4, yaw=0, roll=0):
        self.zoom = zoom
        self.yaw = yaw
        self.roll = roll
    
    def __str__(self):
        return f"({self.zoom}, ({self.yaw}, {self.roll}))"
    
    def set_zoom(self, new_zoom):
        assert new_zoom >= 1
        assert new_zoom <= 25

        while(new_zoom > self.zoom):
            kctl.tap('o')
            self.zoom += 1
            sleep(STANDARD_INPUT_SLEEP)
        while(new_zoom < self.zoom):
            kctl.tap('i')
            self.zoom -= 1
            sleep(STANDARD_INPUT_SLEEP)

    def set_yaw(self, new_yaw):
        diff = (new_yaw - self.yaw) % 8

        if (diff > 0) and (diff <= 4):
            for _ in range(0, diff):
                kctl.tap('.')
                self.yaw = (self.yaw + 1) % 8
                sleep(STANDARD_INPUT_SLEEP)
        elif (diff > 4):
            for _ in range(0, 8 - diff):
                kctl.tap(',')
                self.yaw = (self.yaw - 1) % 8
                sleep(STANDARD_INPUT_SLEEP)

    def set_roll(self, new_roll):
        assert new_roll <= 4
        assert new_roll >= -6

        while(new_roll > self.roll):
            kctl.tap(keyboard.Key.page_up)
            self.roll += 1
            sleep(STANDARD_INPUT_SLEEP)
        while(new_roll < self.roll):
            kctl.tap(keyboard.Key.page_down)
            self.roll -= 1
            sleep(STANDARD_INPUT_SLEEP)

    def set_all(self, zoom=None, yaw=None, roll=None):
        if not type(zoom) == 'NoneType':
            self.set_zoom(zoom)
        if not type(yaw) == 'NoneType':
            self.set_yaw(yaw)
        if not type(roll) == 'NoneType':
            self.set_roll(roll)

    def resync(self, keep_old=False):
        correct_zoom = self.zoom
        self.zoom = 1
        correct_yaw = self.yaw
        self.yaw = 0
        correct_roll = self.roll
        self.roll = 0

        kctl.tap(keyboard.Key.esc)
        sleep(STANDARD_INPUT_SLEEP)
        kctl.tap('r')
        sleep(STANDARD_INPUT_SLEEP)
        kctl.tap(keyboard.Key.enter)
        sleep(STANDARD_INPUT_SLEEP)
        for _ in range(0, 26):
            kctl.tap('i')
            sleep(STANDARD_INPUT_SLEEP)
        kctl.tap('o')
        sleep(STANDARD_INPUT_SLEEP)

        if keep_old:
            self.set_all(correct_zoom, correct_yaw, correct_roll)
        
camera = CameraManager()

The following block defines behaviors for each of the reset types present in the tower gamemode. It will teleport to the correct floor, then automatically orient the camera and click the desired buttons. By modeling the game state, we don't need to normalize our position between actions in order to perform them in an arbitrary order.

In [4]:
warp_dict = {
    'planet_run': 0,
    'hub': 1,
    'field': 2,
    'anti': 3,
    'unnatural': 4,
    'industrial_zone': 5,
    'dark_forest': 6,
    'light_forest': 7,
    'star': 8,
    'solarians': 9,
    'grassland': 10,
    'synthesis': 11,
    'planetoid': 12,
    'floor1': 13,
    'blue': 14,
}

In [20]:
location = 'unknown'

def warp_to(destination, sleep_time=1):
    global location
    if not location == destination:
        index = warp_dict[destination]
        assert index >= 0
        x = 355 + (134 * (index % 10))
        y = 1968 - (134 * math.floor(index / 10))
        lclick(x, y)
        location = destination
        sleep(sleep_time)

def prestige(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('field')
    camera.set_all(1, 6, 0)
    lclick(898, 1437)
    sleep(sleep_time)

def crystallize(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('field')
    camera.set_all(4, 4, 0)
    lclick(1823, 1454)
    sleep(sleep_time)

def grasshop(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('field')
    camera.set_all(4, 2, 0)
    lclick(1086, 1448)
    sleep(sleep_time)

def steelie(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('field')
    camera.set_all(4, 2, 1)
    lclick(1510, 1247)
    sleep(sleep_time)

def rocket_part(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('industrial_zone')
    camera.set_all(4, 0, 0)
    lclick(1022, 1442)
    sleep(sleep_time)

def galactic(sleep_time=DEFAULT_ACTION_TIME):
    global location
    warp_to('industrial_zone')
    camera.set_all(4, 0, 0)
    lclick(1141, 1434)
    location = 'star_platform'
    sleep(sleep_time)

# Manually collect grass from the anti field
def manual_anti(sleep_time = 0.5):
    global location
    warp_to('anti')
    camera.set_yaw(0)

    kctl.press('w')
    sleep(0.1)
    kctl.release('w')
    sleep(0.1)
    kctl.press('a')
    sleep(0.4)
    kctl.release('a')
    sleep(0.1)
    kctl.press('w')
    sleep(0.9)
    kctl.release('w')
    sleep(0.1)
    kctl.press('d')
    sleep(0.8)
    kctl.release('d')
    sleep(0.1)
    kctl.press('s')
    sleep(0.9)
    kctl.release('s')
    location = 'manual_anti'
    sleep(sleep_time)

def liquefy(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('anti')
    camera.set_all(6, 4, 0)
    lclick(260, 1461)
    sleep(sleep_time)

def funify(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('anti')
    camera.set_all(11, 6, 0)
    lclick(865, 1473)
    sleep(sleep_time)

def sacrifice(sleep_time=DEFAULT_ACTION_TIME):
    global location
    warp_to('star_platform') # This isn't intended to actually teleport anywhere in practical use.
    camera.set_all(4, 1, 0)

    # Walk to the sacrifice button because there isn't a good teleport
    kctl.press('w')
    sleep(3)
    kctl.release('w')
    kctl.press('a')
    sleep(4)
    kctl.release('a')
    sleep(0.1)
    location = 'sacrifice'

    lclick(311, 1409)
    sleep(sleep_time)

def fruits(sleep_time=0):
    warp_to('dark_forest')
    camera.set_all(4, 7, -1)
    lclick(1211, 1611) # Select the 10% button
    scroll(1588, 1382, 6) # Scroll to top
    lclick(1588, 1382) # Grass
    lclick(1588, 1484) # TP
    lclick(1588, 1570) # Oil
    scroll(1588, 1382, -3) # Scroll down
    lclick(1588, 1400) # Planetarium
    lclick(1588, 1512) # Cosmic
    scroll(1588, 1382, -3) # Scroll down
    lclick(1588, 1421) # XP
    lclick(1588, 1526) # Fun
    sleep(sleep_time)

# Manually collect grass from the planetoid field
def manual_planetoid(sleep_time = 0.5):
    global location
    warp_to('planetoid')
    camera.set_yaw(0)

    kctl.press('w')
    sleep(0.7)
    kctl.release('w')
    sleep(0.1)
    kctl.press('a')
    sleep(0.4)
    kctl.release('a')
    sleep(0.1)
    kctl.press('w')
    sleep(0.9)
    kctl.release('w')
    sleep(0.1)
    kctl.press('d')
    sleep(0.8)
    kctl.release('d')
    sleep(0.1)
    kctl.press('s')
    sleep(0.9)
    kctl.release('s')
    location = 'manual_planetoid'
    sleep(sleep_time)

def form_ring(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('planetoid')
    camera.set_all(4, 0, 0)
    lclick(545, 1457)
    sleep(sleep_time)

def quadrant(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('planetoid')
    camera.set_all(1, 6, 0)
    lclick(1022, 1421)
    sleep(sleep_time)

def break_ring(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('dark_forest')
    camera.set_all(3, 6, 0)
    lclick(818, 1477)
    sleep(sleep_time)

def planetary(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('planetoid')
    camera.set_all(4, 1, 0)
    lclick(1136, 1430)
    sleep(sleep_time)

def grass_jump(sleep_time=DEFAULT_ACTION_TIME):
    warp_to('unnatural')
    camera.set_all(4, 2, 0)
    lclick(1154, 1472)
    sleep(sleep_time)

def supernova(sleep_time=8):
    warp_to('planetoid')
    camera.set_all(10, 4, 0)
    lclick(960, 1396)
    sleep(sleep_time)

def centralize(repetitions, sleep_time=1):
    warp_to('light_forest')
    camera.set_all(4, 0, 0)
    for _ in range(0, repetitions):
        lclick(1055, 1300)
        lclick(1055, 1390)
        lclick(1055, 1473)
        sleep(sleep_time)

def unlock_solarians(sleep_time=1):
    warp_to('star')
    camera.set_all(4, 6, 0)
    scroll(899, 1259, 100)
    rclick(899, 1259)
    sleep(sleep_time)


In [ ]:
def first_galactic():
    print('first_galactic')
    prestige()
    crystallize()
    grasshop()
    steelie()
    steelie()
    rocket_part()
    galactic(7)

def aghgs_18():
    print('aghgs_18')
    rocket_part()
    manual_anti()
    liquefy()
    funify()
    galactic()
    sacrifice()

def aghgs_30():
    print('aghgs_30')
    rocket_part()
    funify(0)
    galactic()
    fruits()

def planetoid_1():
    print('planetoid_1')
    manual_planetoid()
    form_ring()
    manual_planetoid()
    manual_planetoid()
    form_ring()
    quadrant()
    rocket_part()
    galactic()
    break_ring()

def planetoid_2():
    print('planetoid_2')
    form_ring()
    planetary()
    form_ring()
    grass_jump()
    planetary(0)
    form_ring()
    grass_jump()
    supernova()

def snt5():
    print('snt5')
    galactic(8)
    supernova(4)
    centralize(8)
    unlock_solarians()

def stage_0():
    print('stage 0')
    pass
    # TODO

In [24]:
# Sleep for 1s and normalize the game state to facilitate a handover to the script.
sleep(1)
camera.resync()
location = 'unknown'

# Demo inputs. Can be put in any arbitrary order without worrying about game state.
first_galactic()
aghgs_18()
aghgs_30()
planetoid_1()
planetoid_2()
snt5()
stage_0()

first_galactic
aghgs_18
aghgs_30
planetoid_1
planetoid_2
snt5
stage 0
